In [2]:
import torch
import numpy as np
from transformers import BertTokenizer, BertModel
import chinese_converter
import re

model = BertModel.from_pretrained('bert-base-chinese')
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese', do_lower_case = True)
#MAX_SEQLEN=11000
def make_data():
    data = []
    for i in range(35, 39):
        f = open("training_set/" + str(i) + ".txt", "r")
        lines = f.readlines()
        
        text = lines[1] if len(lines) == 2 else lines[2]
        text_split = re.split('。', text)
        text_split_join = []
        temp = ''
        for i in text_split:
            if len(temp) + len(i) < 512:
                temp += i
            else:
                text_split_join.append(temp)
                temp = ''
        if temp:
            text_split_join.append(temp)
        
        f.close()
        for text in text_split_join:
            label = lines[0]
            text = chinese_converter.to_simplified(text)
            encoded_text = tokenizer(text, padding=True, return_tensors="pt", add_special_tokens=True)
            encoded_text = np.array(encoded_text["input_ids"])[0]#[:MAX_SEQLEN]
            #encoded_text = np.pad(encoded_text, (0, MAX_SEQLEN-len(encoded_text)), mode='constant') 

            if label[0:2] == '[]':
                label_cls = [0] * len(encoded_text)
            else:
                label = label.split(', ')
                label_cls = [0] * len(encoded_text)
                for i in label:
                    i = i.split('\'')
                    i = chinese_converter.to_simplified(i[1])
                    encode_i = tokenizer(i, padding=True, return_tensors="pt", add_special_tokens=True)
                    encode_i = np.array(encode_i['input_ids'][0][1:-1])
                    for j in range(0, len(encoded_text)-len(encode_i)):
                        if (encoded_text[j:j+len(encode_i)] == encode_i).all():
                            for k in range(0, len(encode_i)):
                                label_cls[j+k] = 1
            data.append((torch.tensor(encoded_text), torch.tensor(np.array(label_cls))))
    return data
trainset = make_data()

In [4]:
### from torch.utils.data import DataLoader
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence


def create_mini_batch(samples):
    tokens_tensors = [s[0] for s in samples]
    #segments_tensors = [s[1] for s in samples]
    
    # 測試集有 labels
    
    if samples[0][1] is not None:
        label_ids = [s[1] for s in samples]
    else:
        label_ids = None
    
    # zero pad 到同一序列長度
    tokens_tensors = pad_sequence(tokens_tensors, 
                                  batch_first=True)
    label_ids = pad_sequence(label_ids, 
                                  batch_first=True)
#     segments_tensors = pad_sequence(segments_tensors, 
#                                     batch_first=True)
    
    # attention masks，將 tokens_tensors 裡頭不為 zero padding
    # 的位置設為 1 讓 BERT 只關注這些位置的 tokens
    masks_tensors = torch.zeros(tokens_tensors.shape, 
                                dtype=torch.long)
    masks_tensors = masks_tensors.masked_fill(
        tokens_tensors != 0, 1)
    
    return tokens_tensors, masks_tensors, label_ids

# 初始化一個每次回傳 64 個訓練樣本的 DataLoader
# 利用 `collate_fn` 將 list of samples 合併成一個 mini-batch 是關鍵
BATCH_SIZE = 2
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, 
                         collate_fn=create_mini_batch)

In [ ]:
from transformers import BertForTokenClassification

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

PRETRAINED_MODEL_NAME = "bert-base-chinese"
NUM_LABELS = 2

model = BertForTokenClassification.from_pretrained(
    PRETRAINED_MODEL_NAME, num_labels=NUM_LABELS)

# 訓練模式
model.train()

# 使用 Adam Optim 更新整個分類模型的參數
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)


EPOCHS = 6  # 幸運數字
for epoch in range(EPOCHS):
    temp = 0
    running_loss = 0.0
    for data in trainloader:
        temp += 1
        tokens_tensors, \
        masks_tensors, labels = [t.to(device) for t in data]

        # 將參數梯度歸零
        optimizer.zero_grad()
        
        # forward pass
        outputs = model(input_ids=tokens_tensors,  
                        attention_mask=masks_tensors, 
                        labels=labels)

        loss = outputs[0]
        # backward
        loss.backward()
        optimizer.step()


        # 紀錄當前 batch loss
        running_loss += loss.item()
    print('[epoch %d] loss: %.3f' %
          (epoch + 1, running_loss))

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-c

[epoch 1] loss: 1.650
[epoch 2] loss: 0.466
[epoch 3] loss: 0.310
[epoch 4] loss: 0.282
[epoch 5] loss: 0.226
